In [94]:
import pandas as pd
import numpy as np
import os, sys
from IPython.display import display
from IPython.display import Image as _Imgdis
from numpy import asarray
from matplotlib import pyplot

df = pd.read_csv("Bittrex_BTCUSD_1h.csv", header=1)
df2 = pd.read_csv("Bittrex_ETHUSD_1h.csv",header=1)
df3 = pd.read_csv("Bittrex_LTCUSD_1h.csv",header=1)

df = pd.concat([df, df2, df3])
df.reset_index(drop=True, inplace=True)

df["open_change"] = (df["Open"] / df["Open"].shift(-1)) -1
df["high_change"] = (df["High"] / df["High"].shift(-1)) -1
df["low_change"] = (df["Low"] / df["Low"].shift(-1)) -1
df["close_change"] = (df["Close"] / df["Close"].shift(-1)) - 1

df = df.drop("Unix Timestamp", axis=1)
df = df.drop("Date", axis=1)
df = df.drop("Volume BTC", axis=1)
df = df.drop("Volume ETH", axis=1)
df = df.drop("Volume LTC", axis=1)
df = df.drop("Volume USD", axis=1)
df.drop(df.tail(8).index,inplace=True)

df = df.dropna()

df

,Date,Symbol,Open,High,Low,Close,open_change,high_change,low_change,close_change
0,2020-11-20 07-AM,BTCUSD,18168.59,18390.00,18100.00,18194.27,0.002070,0.011144,0.001045,0.001413
1,2020-11-20 06-AM,BTCUSD,18131.05,18187.32,18081.10,18168.59,-0.003849,-0.002197,0.001187,0.002070
2,2020-11-20 05-AM,BTCUSD,18201.11,18227.37,18059.67,18131.05,0.013638,-0.000418,0.006381,-0.003849
3,2020-11-20 04-AM,BTCUSD,17956.22,18234.99,17945.16,18201.11,-0.000905,0.011471,-0.000496,0.013638
4,2020-11-20 03-AM,BTCUSD,17972.49,18028.19,17954.06,17956.22,-0.000483,-0.001608,0.000862,-0.000905
...,...,...,...,...,...,...,...,...,...,...
88795,2017-07-15 06-AM,LTCUSD,40.05,40.05,40.05,40.05,-0.013547,-0.013547,0.011364,0.000000
88796,2017-07-15 05-AM,LTCUSD,40.60,40.60,39.60,40.05,0.038363,-0.013126,0.036649,-0.013547
88797,2017-07-15 04-AM,LTCUSD,39.10,41.14,38.20,40.60,-0.022744,0.013800,-0.023018,0.038363
88798,2017-07-15 03-AM,LTCUSD,40.01,40.58,39.10,39.10,-0.028412,-0.014570,-0.022500,-0.022744


In [95]:

print(len(df.index))
df.loc[[1]]


88800


,Date,Symbol,Open,High,Low,Close,open_change,high_change,low_change,close_change
1,2020-11-20 06-AM,BTCUSD,18131.05,18187.32,18081.1,18168.59,-0.003849,-0.002197,0.001187,0.00207


In [96]:

X = []
Y = []

changes = df[["Symbol", "open_change", "high_change", "low_change", "close_change"]].to_numpy()


for i in range(0, len(changes) - 20):
    Y.append(changes[i])
    X.append(np.array(changes[i+1:i+21][::-1]))
    
X = np.array(X).reshape(-1, 20, 4)
Y = np.array(Y)
print(X.shape)
print(Y.shape)

(88780, 20, 4)
(88780, 4)


In [97]:
from keras.models import Sequential 
from keras.layers import LSTM

model = Sequential()
model.add(LSTM(4, input_shape=(20, 4)))

model.compile(optimizer="rmsprop", loss="mse")
model.fit(X, Y, batch_size=32, epochs=10)

Epoch 1/10
2775/2775 [==============================] - 18s 6ms/step - loss: 3.3187e-04
Epoch 2/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1618e-04
Epoch 3/10
2775/2775 [==============================] - 17s 6ms/step - loss: 3.1454e-04
Epoch 4/10
2775/2775 [==============================] - 18s 7ms/step - loss: 3.1387e-04
Epoch 5/10
2775/2775 [==============================] - 20s 7ms/step - loss: 3.1340e-04
Epoch 6/10
2775/2775 [==============================] - 18s 6ms/step - loss: 3.1316e-04
Epoch 7/10
2775/2775 [==============================] - 19s 7ms/step - loss: 3.1294e-04
Epoch 8/10
2775/2775 [==============================] - 19s 7ms/step - loss: 3.1274e-04
Epoch 9/10
2775/2775 [==============================] - 18s 6ms/step - loss: 3.1268e-04
Epoch 10/10
2775/2775 [==============================] - 18s 6ms/step - loss: 3.1246e-04


In [108]:
predictions = model.predict(X)
print(predictions.shape)
df1 = pd.DataFrame(predictions, columns=['predictions_open', 'predictions_High', 'predictions_Low', 'predictions_Close'])
#df = pd.concat([df, df1])
df1

(88780, 4)


,predictions_open,predictions_High,predictions_Low,predictions_Close
0,0.001716,0.003322,-0.001957,0.000285
1,-0.004193,0.000073,-0.003311,0.000374
2,0.013296,0.003751,0.007051,-0.000611
3,-0.001308,0.001440,-0.004944,0.000507
4,-0.000793,0.000893,-0.004378,0.000359
...,...,...,...,...
88775,0.005298,0.001673,0.006190,0.000103
88776,-0.008461,0.004237,-0.009079,0.001426
88777,-0.013738,-0.012786,-0.011604,0.003765
88778,-0.041003,-0.024111,-0.016218,0.003232


In [104]:
df1
df1.shape


(88780, 4)